In [2]:
import psycopg2

# Connect to PostgreSQL

conn = psycopg2.connect(
    dbname="GoBike",
    user="postgres",
    password="habiba2004#",
    host="localhost",
    port="5433"
)

In [3]:

cur = conn.cursor()
print("Database connected successfully.")



Database connected successfully.


In [3]:

#Insert rows and values FOR time
sql = "INSERT INTO  dim_time (date, hour, day, day_of_week, month, year) VALUES(%s,%s,%s,%s,%s,%s)"
data = [ 
        ('2023-01-01', 10, 1, 'Sunday', 1, 2023),
        ('2023-01-02', 11, 2, 'Monday', 1, 2023),
        ('2023-01-03', 12, 3, 'Tuesday', 1, 2023)
    ]

cur.executemany(sql, data)
conn.commit()
print("Rows inserted into dim_time table")
print("Number of rows:" ,cur.rowcount)


Rows inserted into dim_time table
Number of rows: 3


In [4]:

#Insert rows and values FOR station
sql = "INSERT INTO dim_station (station_name, latitude, longitude) VALUES (%s,%s,%s)"
data = [
    ("Market St Station", 37.7749, -122.4194),
    ("Central Park Station", 40.7128, -74.0060),
    ("Tower Bridge Station", 51.5074, -0.1278)
]
cur.executemany(sql, data)
conn.commit()
print("Rows inserted into dim_station table")
print("Number of rows:" ,cur.rowcount)
  

Rows inserted into dim_station table
Number of rows: 3


In [9]:
sql = "INSERT INTO dim_user (birth_year, age, gender, user_type) VALUES (%s,%s,%s,%s)"
data = [
    (1990, 33, 'Male', 'Subscriber'),
    (1995, 28, 'Female', 'Customer'),
    (1988, 35, 'Male', 'Subscriber')
]

cur.executemany(sql, data)
conn.commit()
print("Rows inserted successfully into dim_user")

Rows inserted successfully into dim_user


In [29]:
sql = """INSERT INTO fact_trips
(start_time, end_time, duration_sec, bike_id, start_station_id, end_station_id, user_id, time_id) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""

data = [
    ('2023-01-01 10:00:00', '2023-01-01 10:30:00', 1800, 'B001', 1, 2, 1, 1),
    ('2023-01-02 11:15:00', '2023-01-02 11:45:00', 1800, 'B002', 2, 3, 2, 2),
    ('2023-01-03 12:05:00', '2023-01-03 12:50:00', 2700, 'B003', 3, 1, 3, 3)
]

cur.executemany(sql, data)
conn.commit()


In [30]:
print("Rows inserted into fact_trip table")
print("Number of rows:" ,cur.rowcount)

Rows inserted into fact_trip table
Number of rows: 3


In [28]:
conn.rollback()


In [4]:
cur.execute("SELECT * FROM dim_station;")
print(cur.fetchall())

cur.execute("SELECT * FROM dim_user;")
print(cur.fetchall())

cur.execute("SELECT * FROM dim_time;")
print(cur.fetchall())


[(1, 'Market St Station', 37.7749, -122.4194), (2, 'Central Park Station', 40.7128, -74.006), (3, 'Tower Bridge Station', 51.5074, -0.1278)]
[(1, 1990, 33, 'Male', 'Subscriber'), (2, 1995, 28, 'Female', 'Customer'), (3, 1988, 35, 'Male', 'Subscriber'), (4, 1990, 33, 'Male', 'Subscriber'), (5, 1995, 28, 'Female', 'Customer'), (6, 1988, 35, 'Male', 'Subscriber')]
[(1, datetime.date(2023, 1, 1), 10, 1, 'Sunday', 1, 2023), (2, datetime.date(2023, 1, 2), 11, 2, 'Monday', 1, 2023), (3, datetime.date(2023, 1, 3), 12, 3, 'Tuesday', 1, 2023)]


In [1]:
import sys
import psycopg2
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QAction, QTableWidget, QTableWidgetItem, 
    QWidget, QVBoxLayout, QPushButton, QMessageBox
)
from PyQt5.QtCore import Qt

class DatabaseViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Bike Sharing Dashboard")
        self.setGeometry(200, 100, 800, 500)

        # DB connection
        try:
            self.conn = psycopg2.connect(
                dbname="GoBike",
                user="postgres",
                password="habiba2004#",
                host="localhost",
                port="5433"
            )
            self.cur = self.conn.cursor()
        except Exception as e:
            QMessageBox.critical(self, "DB Error", str(e))
            sys.exit(1)

        # Central widget
        self.central_widget = QWidget()
        self.layout = QVBoxLayout()
        self.table = QTableWidget()
        self.refresh_btn = QPushButton("🔄 Refresh Data")
        self.refresh_btn.clicked.connect(self.refresh_data)

        self.layout.addWidget(self.table)
        self.layout.addWidget(self.refresh_btn)
        self.central_widget.setLayout(self.layout)
        self.setCentralWidget(self.central_widget)

        # Menu bar
        menubar = self.menuBar()
        viewMenu = menubar.addMenu("📊 View")

        users_action = QAction("Users", self)
        users_action.triggered.connect(lambda: self.load_table("dim_user"))
        stations_action = QAction("Stations", self)
        stations_action.triggered.connect(lambda: self.load_table("dim_station"))
        trips_action = QAction("Trips", self)
        trips_action.triggered.connect(lambda: self.load_table("fact_trips"))

        viewMenu.addAction(users_action)
        viewMenu.addAction(stations_action)
        viewMenu.addAction(trips_action)

        self.current_table = "dim_user"
        self.load_table(self.current_table)

    def load_table(self, table_name):
        try:
            self.cur.execute(f"SELECT * FROM {table_name}")
            rows = self.cur.fetchall()
            cols = [desc[0] for desc in self.cur.description]

            self.table.setRowCount(len(rows))
            self.table.setColumnCount(len(cols))
            self.table.setHorizontalHeaderLabels(cols)

            for i, row in enumerate(rows):
                for j, val in enumerate(row):
                    self.table.setItem(i, j, QTableWidgetItem(str(val)))

            self.table.resizeColumnsToContents()
            self.current_table = table_name
        except Exception as e:
            QMessageBox.critical(self, "Error", str(e))

    def refresh_data(self):
        self.load_table(self.current_table)

    def closeEvent(self, event):
        if self.conn:
            self.cur.close()
            self.conn.close()
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    viewer = DatabaseViewer()
    viewer.show()
    sys.exit(app.exec_())



SystemExit: 0

C:\Users\habib\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
